In [1]:
import os
import sys

# Current notebook folder: win_tracker/soccer
notebook_dir = os.getcwd()

# Project root for imports: win_tracker/
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

# Add to sys.path if not already there
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added to sys.path:", project_root)


Project root added to sys.path: /Users/Jake/Documents/GitHub/all-sports-win-tracker/win_tracker


In [3]:
import time
import cfbd
import config
from cfbd.models.division_classification import DivisionClassification
from cfbd.models.game import Game
from cfbd.models.season_type import SeasonType
from cfbd.rest import ApiException
from pprint import pprint
from pyspark.sql import SparkSession

from pyspark.sql.functions import when, col, date_format
from func import write_single_csv

In [4]:
spark = SparkSession.builder.appName("PandasToSpark").getOrCreate()

In [5]:


# Defining the host is optional and defaults to https://api.collegefootballdata.com
# See configuration.py for a list of all supported configuration parameters.
configuration = cfbd.Configuration(
    host = "https://api.collegefootballdata.com"
)

# The client must configure the authentication and authorization parameters
# in accordance with the API server security policy.
# Examples for each auth method are provided below, use the example that
# satisfies your auth use case.

# # Configure Bearer authorization: apiKey
configuration = cfbd.Configuration(
    access_token = "CytOpJZNN7WKpNRqmTt8fTOEH4IunpazKE6OyLgLYRd/+gG8nbibFM+r2SVix+/a"
)

all_games = []
for year in range(2020, 2026):
    # Enter a context with an instance of the API client
    with cfbd.ApiClient(configuration) as api_client:
        # Create an instance of the API class
        api_instance = cfbd.GamesApi(api_client)
        year = year # int | Required year filter (except when id is specified) (optional)
        # week = 56 # int | Optional week filter (optional)
        # # season_type = cfbd.SeasonType() # SeasonType | Optional season type filter (optional)
        # classification = cfbd.DivisionClassification() # DivisionClassification | Optional division classification filter (optional)
        team = config.cfb_team_to_filter # str | Optional team filter (optional)
        # home = 'home_example' # str | Optional home team filter (optional)
        # away = 'away_example' # str | Optional away team filter (optional)
        # conference = 'conference_example' # str | Optional conference filter (optional)
        # id = 56 # int | Game id filter to retrieve a single game (optional)
        try:
            api_response = api_instance.get_games(year=year, team=team)
            all_games.extend(api_response)
            #print("The response of GamesApi->get_games:\n")
            pprint(api_response)
        except Exception as e:
            print("Exception when calling GamesApi->get_games: %s\n" % e)

[Game(id=401247296, season=2020, week=8, season_type=<SeasonType.REGULAR: 'regular'>, start_date=datetime.datetime(2020, 10, 24, 19, 30, tzinfo=datetime.timezone.utc), start_time_tbd=False, completed=True, neutral_site=False, conference_game=True, attendance=900, venue_id=3907, venue='Ross-Ade Stadium', home_id=2509, home_team='Purdue', home_conference='Big Ten', home_classification=<DivisionClassification.FBS: 'fbs'>, home_points=24, home_line_scores=[7, 7, 0, 10], home_postgame_win_probability=0.14702423947056187, home_pregame_elo=1492, home_postgame_elo=1504, away_id=2294, away_team='Iowa', away_conference='Big Ten', away_classification=<DivisionClassification.FBS: 'fbs'>, away_points=20, away_line_scores=[0, 17, 0, 3], away_postgame_win_probability=0.8529757605294381, away_pregame_elo=1783, away_postgame_elo=1771, excitement_index=9.3116167939, highlights='https://www.youtube.com/watch?v=gPRZCy6CXYs', notes=None),
 Game(id=401247303, season=2020, week=9, season_type=<SeasonType.REG

In [6]:
game_list = []
for game_instance in api_response:
    game_list.append(game_instance.to_dict())

print(game_list)


[{'id': 401752799, 'season': 2025, 'week': 1, 'seasonType': <SeasonType.REGULAR: 'regular'>, 'startDate': datetime.datetime(2025, 8, 30, 22, 0, tzinfo=datetime.timezone.utc), 'startTimeTBD': False, 'completed': True, 'neutralSite': False, 'conferenceGame': False, 'venueId': 3793, 'venue': 'Kinnick Stadium', 'homeId': 2294, 'homeTeam': 'Iowa', 'homeConference': 'Big Ten', 'homeClassification': <DivisionClassification.FBS: 'fbs'>, 'homePoints': 34, 'homeLineScores': [3, 14, 7, 10], 'homePostgameWinProbability': 0.9937663078308105, 'homePregameElo': 1682, 'homePostgameElo': 1682, 'awayId': 399, 'awayTeam': 'UAlbany', 'awayConference': 'CAA', 'awayClassification': <DivisionClassification.FCS: 'fcs'>, 'awayPoints': 7, 'awayLineScores': [0, 7, 0, 0], 'awayPostgameWinProbability': 0.006233692169189453, 'excitementIndex': 3.706247041004862, 'highlights': '', 'attendance': None, 'awayPregameElo': None, 'awayPostgameElo': None, 'notes': None}, {'id': 401752822, 'season': 2025, 'week': 2, 'season

In [7]:
game_list = [g.to_dict() for g in all_games]
print({g.season for g in all_games})
print(len(game_list), "total games")
print(game_list)
print(game_list[0].keys())

{2020, 2021, 2022, 2023, 2024, 2025}
74 total games
[{'id': 401247296, 'season': 2020, 'week': 8, 'seasonType': <SeasonType.REGULAR: 'regular'>, 'startDate': datetime.datetime(2020, 10, 24, 19, 30, tzinfo=datetime.timezone.utc), 'startTimeTBD': False, 'completed': True, 'neutralSite': False, 'conferenceGame': True, 'attendance': 900, 'venueId': 3907, 'venue': 'Ross-Ade Stadium', 'homeId': 2509, 'homeTeam': 'Purdue', 'homeConference': 'Big Ten', 'homeClassification': <DivisionClassification.FBS: 'fbs'>, 'homePoints': 24, 'homeLineScores': [7, 7, 0, 10], 'homePostgameWinProbability': 0.14702423947056187, 'homePregameElo': 1492, 'homePostgameElo': 1504, 'awayId': 2294, 'awayTeam': 'Iowa', 'awayConference': 'Big Ten', 'awayClassification': <DivisionClassification.FBS: 'fbs'>, 'awayPoints': 20, 'awayLineScores': [0, 17, 0, 3], 'awayPostgameWinProbability': 0.8529757605294381, 'awayPregameElo': 1783, 'awayPostgameElo': 1771, 'excitementIndex': 9.3116167939, 'highlights': 'https://www.youtube

In [8]:


keys_to_drop = ['seasonType', 'homeClassification', "awayClassification"]
filtered_list = [
    {k: v for k, v in game.items() if k not in keys_to_drop}
    for game in game_list
]
print(filtered_list[0].keys())

game_list_str = [{k: str(v) if v is not None else None for k, v in d.items()} for d in filtered_list]


df = spark.createDataFrame(game_list_str)

# Inspect
print(df.count())
df.show(5, truncate=False)


dict_keys(['id', 'season', 'week', 'startDate', 'startTimeTBD', 'completed', 'neutralSite', 'conferenceGame', 'attendance', 'venueId', 'venue', 'homeId', 'homeTeam', 'homeConference', 'homePoints', 'homeLineScores', 'homePostgameWinProbability', 'homePregameElo', 'homePostgameElo', 'awayId', 'awayTeam', 'awayConference', 'awayPoints', 'awayLineScores', 'awayPostgameWinProbability', 'awayPregameElo', 'awayPostgameElo', 'excitementIndex', 'highlights', 'notes'])


25/11/09 16:30:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


74
+----------+--------------+------+--------------+----------+---------------+--------------------------+--------------+--------------+---------+--------------+---------------+-------------------------------------------+--------------+------+--------------+----------+---------------+--------------------------+--------------+----------+---------+-----------+-----+------+-------------------------+------------+-----------------------+-------+----+
|attendance|awayConference|awayId|awayLineScores|awayPoints|awayPostgameElo|awayPostgameWinProbability|awayPregameElo|awayTeam      |completed|conferenceGame|excitementIndex|highlights                                 |homeConference|homeId|homeLineScores|homePoints|homePostgameElo|homePostgameWinProbability|homePregameElo|homeTeam  |id       |neutralSite|notes|season|startDate                |startTimeTBD|venue                  |venueId|week|
+----------+--------------+------+--------------+----------+---------------+--------------------------+

In [9]:
# Change data types as needed
df = (
    df.withColumn("season", col("season").cast("int"))
    .withColumn("week", col("week").cast("int"))
    .withColumn("awayPoints", col("awayPoints").cast("int"))
    .withColumn("homePoints", col("homePoints").cast("int")).withColumn("startDate", col("startDate").cast("timestamp"))
)

In [10]:
# Create winner column

df = df.withColumn("Winner", when(col("homePoints") > col("awayPoints"), col("homeTeam"))
                            .when(col("homePoints") < col("awayPoints"), col("awayTeam"))
                            .otherwise("Tie"))

df.show()

+----------+--------------+------+--------------+----------+---------------+--------------------------+--------------+--------------+---------+--------------+---------------+--------------------+--------------+------+--------------+----------+---------------+--------------------------+--------------+------------+---------+-----------+-----+------+-------------------+------------+--------------------+-------+----+------------+
|attendance|awayConference|awayId|awayLineScores|awayPoints|awayPostgameElo|awayPostgameWinProbability|awayPregameElo|      awayTeam|completed|conferenceGame|excitementIndex|          highlights|homeConference|homeId|homeLineScores|homePoints|homePostgameElo|homePostgameWinProbability|homePregameElo|    homeTeam|       id|neutralSite|notes|season|          startDate|startTimeTBD|               venue|venueId|week|      Winner|
+----------+--------------+------+--------------+----------+---------------+--------------------------+--------------+--------------+-------

In [11]:
# Create day of week column and date column
df = df.withColumn("day_of_week_name", date_format("startDate", "EEEE")).withColumn("game_date", date_format("startDate", "yyyy-MM-dd"))
df.show()

+----------+--------------+------+--------------+----------+---------------+--------------------------+--------------+--------------+---------+--------------+---------------+--------------------+--------------+------+--------------+----------+---------------+--------------------------+--------------+------------+---------+-----------+-----+------+-------------------+------------+--------------------+-------+----+------------+----------------+----------+
|attendance|awayConference|awayId|awayLineScores|awayPoints|awayPostgameElo|awayPostgameWinProbability|awayPregameElo|      awayTeam|completed|conferenceGame|excitementIndex|          highlights|homeConference|homeId|homeLineScores|homePoints|homePostgameElo|homePostgameWinProbability|homePregameElo|    homeTeam|       id|neutralSite|notes|season|          startDate|startTimeTBD|               venue|venueId|week|      Winner|day_of_week_name| game_date|
+----------+--------------+------+--------------+----------+---------------+--------

In [12]:
cfb_data_final = df.select("season", "week", "awayTeam", "homeTeam", "awayPoints", "homePoints", "Winner", "game_date", "day_of_week_name")
cfb_data_final.show()

+------+----+--------------+------------+----------+----------+------------+----------+----------------+
|season|week|      awayTeam|    homeTeam|awayPoints|homePoints|      Winner| game_date|day_of_week_name|
+------+----+--------------+------------+----------+----------+------------+----------+----------------+
|  2020|   8|          Iowa|      Purdue|        20|        24|      Purdue|2020-10-24|        Saturday|
|  2020|   9|  Northwestern|        Iowa|        21|        20|Northwestern|2020-10-31|        Saturday|
|  2020|  10|Michigan State|        Iowa|         7|        49|        Iowa|2020-11-07|        Saturday|
|  2020|  11|          Iowa|   Minnesota|        35|         7|        Iowa|2020-11-13|          Friday|
|  2020|  12|          Iowa|  Penn State|        41|        21|        Iowa|2020-11-21|        Saturday|
|  2020|  13|      Nebraska|        Iowa|        20|        26|        Iowa|2020-11-27|          Friday|
|  2020|  14|          Iowa|    Illinois|        35|   

In [ ]:
write_single_csv(cfb_data_final, "win_tracker/cfb/cfb_data.csv")